In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import csv
import re
df = pd.read_csv("/content/drive/MyDrive/project/chatbot/dataset/total.csv", encoding='utf-8', error_bad_lines=False, skipinitialspace=True)
labels = df['label'].unique()
for i, v in enumerate(labels):
  for index, value in enumerate(df['label']):
     if value == v:
       df['label'][index] = i
df.sort_values('label', inplace=True)
df.reset_index(drop=True, inplace=True)
df

<ipython-input-3-f9d7d18cf601>:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv("/content/drive/MyDrive/project/chatbot/dataset/total.csv", encoding='utf-8', error_bad_lines=False, skipinitialspace=True)


,Q,A,label
0,영화관 가는거 좋아해?,응 훨씬 몰입이 잘되는거 같아 집에서 보는 것 보다,0
1,그래서 이터널스 후기 몇 점?,흠... 제발 기대가 커서 그런가 ㅠ,0
2,팝핀 댄스 같이 배우장 고고,난 몸치라... 가능할지 모르겠네,0
3,엉 스우파 나오신 분...!,진짜... 시대에 뒤떨어진 노인 같다 나...,0
4,이야 노제 모르는 사람이 있다니,유튭으로 짧게만 봐서 잘 모름 키키,0
...,...,...,...
193264,아 그거 내가 말렸거든,그 때 400불 정도 였는데,20
193265,지금 천슬라 될 것 같네,아 그니까 내가 투자하자고 했잖아,20
193266,아니 돈을 줘야 사는데 돈을 안 줬어,아니 그건 알아서 마련해야지,20
193267,나는 용기있으니까 정보 좀 나눠줘,친구따라 강남 가는게 아니라 서울역 갈지도 몰라,20


In [ ]:
# df = df[['question', 'clue', 'label']]
# df.columns = ['Q','A','label']
# df

KeyError: ignored

In [ ]:
num_of_rows = df.shape[0]
print("스킵되지 않은 행의 수:", num_of_rows)

스킵되지 않은 행의 수: 193269


In [ ]:
df = df.dropna()

In [ ]:
df.isnull().sum()

Q        0
A        0
label    0
dtype: int64

In [ ]:
!pip install git+https://github.com/huggingface/transformers
!pip install pytorch-lightning

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-6y_buvjq
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-6y_buvjq
  Resolved https://github.com/huggingface/transformers to commit 093848d3ccf3884caf048718b6bae833da0edb94
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 78.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.7 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.35.0.dev0-py3-none-any.whl size=7817578 sha256=923ed35b497e40468b13caa3d38c3710135aa25f12d8b9dc8693fc444ea8caf4
  Stored in directory: /tmp/pip-ephem-wheel-cache-ayew

In [ ]:
import torch
from transformers import GPT2LMHeadModel
from transformers import PreTrainedTokenizerFast

In [ ]:
Chatbot_Data = df
print(len(Chatbot_Data))
Chatbot_Data.head()

193256


,Q,A,label
0,영화관 가는거 좋아해?,응 훨씬 몰입이 잘되는거 같아 집에서 보는 것 보다,0
1,그래서 이터널스 후기 몇 점?,흠... 제발 기대가 커서 그런가 ㅠ,0
2,팝핀 댄스 같이 배우장 고고,난 몸치라... 가능할지 모르겠네,0
3,엉 스우파 나오신 분...!,진짜... 시대에 뒤떨어진 노인 같다 나...,0
4,이야 노제 모르는 사람이 있다니,유튭으로 짧게만 봐서 잘 모름 키키,0


In [ ]:
#토큰 정의
Q_TKN = "<usr>"  #사용자 질문 토큰
A_TKN = "<sys>"  #시스템 답변 토큰
BOS = '</s>'    #문장 시작 토큰
EOS = '</s>'    #문장 종료 토큰
MASK = '<unused0>'  #마스크 토큰
SENT = '<unused1>'  #미사용 토큰
PAD = '<pad>'       #패딩 토큰

# 허깅페이스 transformers 에 등록된 사전 학습된 koGTP2 토크나이저를 가져온다.
koGPT2_TOKENIZER = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2", bos_token=BOS, eos_token=EOS, unk_token="<unk>", pad_token=PAD, mask_token=MASK,)
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
model.resize_token_embeddings(len(koGPT2_TOKENIZER))

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


Embedding(51200, 768)

In [ ]:
from pytorch_lightning import Trainer  #학습 과정을 다루는 라이브러리
from pytorch_lightning.callbacks import ModelCheckpoint  #모델 체크포인트를 학습 중에 저장하기 위함
from pytorch_lightning.core import LightningModule     #학습 및 검증 단계 정의
from torch.utils.data import DataLoader, Dataset      #데이터를 효율적으로 로딩하고 처리
from transformers.optimization import AdamW, get_cosine_schedule_with_warmup   #fine-tuning하기 위한 옵티마이저, 학습률 스케줄링 구성요소
import re  #정규 표현식 다루기 위한 모듈

In [ ]:
# 챗봇 데이터를 처리하는 클래스를 만든다.
class ChatbotDataset(Dataset):
    def __init__(self, chats, max_len=40):  # 데이터셋의 전처리를 해주는 부분
        self._data = chats
        self.max_len = max_len
        self.q_token = Q_TKN
        self.a_token = A_TKN
        self.sent_token = SENT
        self.eos = EOS
        self.mask = MASK
        self.tokenizer = koGPT2_TOKENIZER

    def __len__(self):  # chatbotdata 의 길이를 리턴한다.
        return len(self._data)

    def __getitem__(self, idx):  # 로드한 챗봇 데이터를 차례차례 DataLoader로 넘겨주는 메서드
        turn = self._data.iloc[idx]
        q = turn["Q"]  # 질문을 가져온다.
        q = re.sub(r"([^A-Za-z0-9가-힣\s])", r" ", q)  # 구둣점들을 제거한다.

        a = turn["A"]  # 답변을 가져온다.
        a = re.sub(r"([^A-Za-z0-9가-힣\s])", r" ", a)  # 구둣점들을 제거한다.

        q_toked = self.tokenizer.tokenize(self.q_token + q + self.sent_token)
        q_len = len(q_toked)

        a_toked = self.tokenizer.tokenize(self.a_token + a + self.eos)
        a_len = len(a_toked)

        #질문의 길이가 최대길이보다 크면
        if q_len > self.max_len:
            a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        #질문의 길이 + 답변의 길이가 최대길이보다 크면
        if q_len + a_len > self.max_len:
            a_len = self.max_len - q_len        #답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:       #질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_toked = q_toked[-(int(self.max_len / 2)) :]   #질문길이를 최대길이의 반으로
                q_len = len(q_toked)
                a_len = self.max_len - q_len              #답변의 길이를 최대길이 - 질문길이
            a_toked = a_toked[:a_len]
            a_len = len(a_toked)

        # 답변 labels = [mask, mask, ...., mask, ..., <bos>,..답변.. <eos>, <pad>....]
        labels = [self.mask,] * q_len + a_toked[1:]

        # mask = 질문길이 0 + 답변길이 1 + 나머지 0
        mask = [0] * q_len + [1] * a_len + [0] * (self.max_len - q_len - a_len)
        # 답변 labels을 index 로 만든다.
        labels_ids = self.tokenizer.convert_tokens_to_ids(labels)
        # 최대길이만큼 PADDING
        while len(labels_ids) < self.max_len:
            labels_ids += [self.tokenizer.pad_token_id]

        # 질문 + 답변을 index 로 만든다.
        token_ids = self.tokenizer.convert_tokens_to_ids(q_toked + a_toked)
        # 최대길이만큼 PADDING
        while len(token_ids) < self.max_len:
            token_ids += [self.tokenizer.pad_token_id]

        #질문+답변, 마스크, 답변
        return (token_ids, np.array(mask), labels_ids)

In [ ]:
#데이터의 batch를 처리하는 함수 정의
##데이터 batch의 요소들을 추출하여 텐서로 변환하고, GPU 메모리로 이동시키는 역할을 함
def collate_batch(batch):
  #data, mask, label 변수 초기화
    data = [item[0] for item in batch]
    mask = [item[1] for item in batch]
    label = [item[2] for item in batch]
    return torch.LongTensor(data).cuda(), torch.LongTensor(mask).cuda(), torch.LongTensor(label).cuda() #텐서로 변환하고 CUDA로 이동

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#train 데이터셋 정의
train_set = ChatbotDataset(Chatbot_Data, max_len=40)
#윈도우 환경에서 num_workers 는 무조건 0으로 지정, 리눅스에서는 2
#torch.utils.data.DataLoader는 PyTorch 라이브러리에서 제공하는 데이터 로딩 유틸리티입니다.
#이를 사용하면 데이터셋을 미니배치(mini-batch)로 나누고 병렬적으로 데이터를 로드하여 모델 학습 시에 효율적으로 데이터를 공급할 수 있습니다.
train_dataloader = torch.utils.data.DataLoader(train_set, batch_size=32, num_workers=0, shuffle=True, collate_fn=collate_batch)

In [ ]:
model.to(device)
model.train()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

In [ ]:
learning_rate = 3e-5   #학습률(모델이 갱신되는 정도 조절)
#torch.nn.CrossEntropyLoss는 PyTorch에서 분류 문제를 위한 손실 함수 중 하나입니다.
#이 함수는 주로 다중 클래스 분류 작업에서 사용되며, 모델의 출력값과 실제 타깃 레이블 간의 차이를 측정하여 모델을 학습시킵니다.
#reduction 매개변수는 손실의 감소 방식을 지정하는 역할을 합니다. 이 매개변수에는 다음과 같은 옵션이 있습니다:
#"none": 각 샘플에 대한 손실을 계산하고 반환합니다. 즉, 각 데이터 포인트마다 개별적인 손실값을 가집니다.
#"mean": 모든 샘플의 손실을 평균하여 하나의 스칼라 값으로 반환합니다.
#"sum": 모든 샘플의 손실을 합하여 하나의 스칼라 값으로 반환합니다.
#reduction="none"을 설정하면, 각 데이터 포인트의 손실이 개별적으로 유지되므로, 예를 들어 배치 내의 각 데이터 포인트마다 어떤 클래스를 잘못 예측했는지에 대한 정보를 보다 상세하게 얻을 수 있습니다
criterion = torch.nn.CrossEntropyLoss(reduction="none")  #손실함수 설정
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  #옵티마이저 설정

epoch = 5 #학습 에폭(데이터셋을 몇 번 반복하여 학습시킬지)
Sneg = -1e18 #음수 무한대 값을 나타내는 변수

In [ ]:
print ("start")  #학습 시작 메시지
for epoch in range(epoch):   #에폭 수 만큼 반복하며 학습
    for batch_idx, samples in enumerate(train_dataloader):  #데이터로더에서 미니배치 순회
        optimizer.zero_grad()    #그래디언트 초기화
        token_ids, mask, label = samples  #미니배치로부터 토큰ID, 마스크, 라벨 가져옴
        out = model(token_ids)      #모델의 예측 결과 얻음
        out = out.logits      #모델 출력을 로짓으로 변환
        mask_3d = mask.unsqueeze(dim=2).repeat_interleave(repeats=out.shape[2], dim=2)  #마스크를 로짓 형태에 맞게 확장
        mask_out = torch.where(mask_3d == 1, out, Sneg * torch.ones_like(out))  #로짓에 마스크 적용
        loss = criterion(mask_out.transpose(2, 1), label)  #마스크가 적용된 로짓과 실제 라벨 간의 손실 계산
        avg_loss = loss.sum() / mask.sum()   #손실을 평균화
        avg_loss.backward()  #역전파 수행하여 그래디언트 계산

        optimizer.step()   #옵티마이저로 모델 파라미터 업데이트
    print(f'Epoch {epoch} 완료')
print ("end")  #학습 종료 메시지

<ipython-input-12-4341cdaa9e3b>:8: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  return torch.LongTensor(data).cuda(), torch.LongTensor(mask).cuda(), torch.LongTensor(label).cuda() #텐서로 변환하고 CUDA로 이동
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


start
Epoch 0 완료
Epoch 1 완료
Epoch 2 완료
Epoch 3 완료
Epoch 4 완료
end


In [ ]:
#챗봇 테스트 함수
def after_gpt_test(q):
# 맥락 안에서 모델을 사용하여 추론을 수행하고 있습니다. 이렇게 함으로써 연산에 대한 그래디언트 계산이 비활성화되므로, 모델을 평가할 때나 추론을 할 때 불필요한 계산을 줄일 수 있습니다.
    with torch.no_grad():
      q = q.strip()
      a = ""
      while 1:
          input_ids = torch.LongTensor(koGPT2_TOKENIZER.encode(Q_TKN + q + SENT + A_TKN + a)).cuda().unsqueeze(dim=0)  #입력 문장을 토크나이징하고 GPU에 전송
          pred = model(input_ids)   #모델에 입력을 주어 대답 생성
          pred = pred.logits.cpu()
          gen = koGPT2_TOKENIZER.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().numpy().tolist())[-1]  #모델 출력에서 다음 토큰을 선택해 대답 생성함
          if gen == EOS:  #대답 토큰이 종료 토큰과 일치될 때(대답 생성이 종료될 때)
              break     #루프 빠져나옴
          a += gen.replace("▁", " ")
      print("Chatbot > {}".format(a.strip()))   #생성된 대답 출력
      print()
    return

In [ ]:
text = '오늘 영화보러 갈래? '

after_gpt_test(text)

Chatbot > 어디서 보려고



In [ ]:
text = "저녁메뉴 추천 좀 해줄래?"

after_gpt_test(text)

Chatbot > 저녁메뉴는 뭐야



In [ ]:
torch.save(model, '/content/drive/MyDrive/project/chatbot/dataset/model2.pt')

In [ ]:
import joblib
joblib.dump(koGPT2_TOKENIZER, "/content/drive/MyDrive/project/chatbot/dataset/tokenizer.pkl")

['/content/drive/MyDrive/project/chatbot/dataset/tokenizer.pkl']